# 📘 Optical Properties Post-Processing from VASP OUTCAR

---

## 1️⃣ Overview

This notebook extracts and post-processes the **frequency-dependent dielectric function** from a VASP `OUTCAR` file and computes:

- Direction-dependent refractive indices: \( n_x, n_y, n_z \)
- Birefringence:
  
  \[
  \Delta n = \max(n_x, n_y, n_z) - \min(n_x, n_y, n_z)
  \]

- Smoothed optical spectra using cubic spline interpolation
- Wavelength-dependent plots in the 450–3000 nm range
- Exported numerical optical data for further analysis

The workflow is particularly useful for anisotropic materials such as nonlinear optical crystals.

---

## 2️⃣ Physical Background

### Complex Dielectric Function

VASP provides the frequency-dependent dielectric function:

\[
\varepsilon(\omega) = \varepsilon_1(\omega) + i \varepsilon_2(\omega)
\]

where:

- \( \varepsilon_1 \) is the real part  
- \( \varepsilon_2 \) is the imaginary part  

---

### Refractive Index from Dielectric Function

The refractive index is computed from the complex dielectric function as:

\[
n(\omega) =
\sqrt{
\frac{
\sqrt{\varepsilon_1^2 + \varepsilon_2^2}
+
\varepsilon_1
}{2}
}
\]

This expression ensures the physically meaningful positive branch of the refractive index.

---

### Energy–Wavelength Conversion

Photon energy and wavelength are related by:

\[
\lambda \, (\text{nm}) = \frac{1239.84193}{E \, (\text{eV})}
\]

This conversion allows visualization of optical spectra in the wavelength domain, which is more intuitive for comparison with experimental laser wavelengths (e.g., 532 nm, 1064 nm).

---

## 3️⃣ Features of This Notebook

✔ Automatic extraction of dielectric tensor data from `OUTCAR`  
✔ Conversion from energy (eV) to wavelength (nm)  
✔ Filtering of visible and infrared region (≥ 450 nm)  
✔ Smooth cubic spline interpolation for publication-quality curves  
✔ Export of tabulated optical data  
✔ High-resolution plots suitable for journal submission  

---




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

# ============================================================
# 1. Reading Dielectric Function from OUTCAR
# ============================================================

def read_dielectric_function(outcar_file):
    """
    Extracts the real and imaginary dielectric tensor components
    from a VASP OUTCAR file.

    Parameters
    ----------
    outcar_file : str
        Path to OUTCAR file.

    Returns
    -------
    energy : ndarray
        Energy grid in eV.
    real_eps : ndarray
        Real dielectric tensor (ε1_x, ε1_y, ε1_z).
    imag_eps : ndarray
        Imaginary dielectric tensor (ε2_x, ε2_y, ε2_z).
    """

    with open(outcar_file, 'r') as f:
        lines = f.readlines()

    real_start = None
    imag_start = None

    for i, line in enumerate(lines):
        if "REAL DIELECTRIC FUNCTION (independent particle, no local field effects) density-density" in line:
            real_start = i + 3
        elif "IMAGINARY DIELECTRIC FUNCTION (independent particle, no local field effects) density-density" in line:
            imag_start = i + 3

    def read_block(start_idx):
        data = []
        i = start_idx
        while i < len(lines) and not lines[i].strip().startswith('-----'):
            parts = lines[i].split()
            if len(parts) == 7:
                try:
                    data.append([float(x) for x in parts])
                except ValueError:
                    pass
            i += 1
        return np.array(data)

    if real_start is None or imag_start is None:
        raise ValueError("Dielectric function sections not found in OUTCAR.")

    real_data = read_block(real_start)
    imag_data = read_block(imag_start)

    energy = real_data[:, 0]
    real_eps = real_data[:, 1:4]
    imag_eps = imag_data[:, 1:4]

    return energy, real_eps, imag_eps


# ============================================================
# 2. Optical Property Calculations
# ============================================================

def calculate_refractive_indices(real_eps, imag_eps):
    """
    Compute refractive index from complex dielectric function.
    """
    return np.sqrt((np.sqrt(real_eps**2 + imag_eps**2) + real_eps) / 2)


def energy_to_wavelength(energy_eV):
    """
    Convert photon energy (eV) to wavelength (nm).
    """
    return 1239.84193 / energy_eV


def filter_by_wavelength(energy, *arrays, min_wavelength=450):
    """
    Filter arrays for wavelengths >= min_wavelength.
    """
    wavelength = energy_to_wavelength(energy)
    mask = (wavelength >= min_wavelength) & np.isfinite(wavelength) & (energy > 0)

    filtered = [energy[mask]]
    for arr in arrays:
        filtered.append(arr[mask])

    return tuple(filtered)


# ============================================================
# 3. Data Saving
# ============================================================

def save_all_data(energy, n, delta_n, filename):
    """
    Save processed optical data to file.
    """
    wavelength = energy_to_wavelength(energy)

    header = "Wavelength (nm)\tEnergy (eV)\tn_x\tn_y\tn_z\tBirefringence (Δn)"

    data = np.column_stack((
        wavelength,
        energy,
        n[:, 0],
        n[:, 1],
        n[:, 2],
        delta_n
    ))

    np.savetxt(filename, data, header=header, delimiter='\t', fmt='%.6f')


# ============================================================
# 4. Plotting Functions
# ============================================================

def plot_refractive_indices(energy, n, save_path=None, material_name=""):
    """
    Plot n_x, n_y, n_z vs wavelength using cubic spline smoothing.
    """

    wavelength = energy_to_wavelength(energy)
    wl_plot = np.linspace(450, 3000, 600)

    plt.figure(figsize=(8,6))

    for i, label in enumerate(['n_x', 'n_y', 'n_z']):
        cs = CubicSpline(wavelength[::-1], n[::-1, i])
        plt.plot(wl_plot, cs(wl_plot), label=label)

    plt.xlim(450, 3000)
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Refractive Index (n)")
    plt.title(f"Refractive Indices ({material_name})")
    plt.grid(True)
    plt.legend()

    if save_path:
        plt.savefig(save_path, dpi=500)

    plt.show()


def plot_birefringence(energy, delta_n, save_path=None,
                       highlight_wavelength=None, material_name=""):
    """
    Plot birefringence vs wavelength.
    """

    wavelength = energy_to_wavelength(energy)
    wl_plot = np.linspace(450, 3000, 800)

    cs = CubicSpline(wavelength[::-1], delta_n[::-1])

    plt.figure(figsize=(8,6))
    plt.plot(wl_plot, cs(wl_plot))

    if highlight_wavelength:
        val = cs(highlight_wavelength)
        plt.scatter(highlight_wavelength, val)
        plt.annotate(f"Δn = {val:.4f} at {highlight_wavelength} nm",
                     (highlight_wavelength, val))

    plt.xlim(450, 3000)
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Birefringence Δn")
    plt.title(f"Birefringence ({material_name})")
    plt.grid(True)

    if save_path:
        plt.savefig(save_path, dpi=500)

    plt.show()


# ============================================================
# 5. Main Execution
# ============================================================

def main():

    outcar_file = "OUTCAR"
    material_name = "ABX4"

    energy, real_eps, imag_eps = read_dielectric_function(outcar_file)

    n = calculate_refractive_indices(real_eps, imag_eps)
    delta_n = np.max(n, axis=1) - np.min(n, axis=1)

    energy, n, delta_n = filter_by_wavelength(energy, n, delta_n)

    plot_refractive_indices(
        energy, n,
        save_path="refractive_indices.png",
        material_name=material_name
    )

    plot_birefringence(
        energy, delta_n,
        save_path="birefringence.png",
        highlight_wavelength=532,
        material_name=material_name
    )

    save_all_data(energy, n, delta_n, "optical_data.txt")

    print("✅ Data processing complete!")
    print("Plots and optical data saved successfully.")


if __name__ == "__main__":
    main()
